In [ ]:
import numpy as np 
import pandas as pd
import os

from tabula import read_pdf
from tabulate import tabulate

In [ ]:
# Define Dir with working raw datasets
os.getcwd()

pdf_dir = 'Raw Data/'
print(os.listdir(pdf_dir))

output_dir = 'Cleaned Data/'
print(os.listdir(output_dir))


In [ ]:
# Test on specific states
tables_pg3 = read_pdf(os.path.join(pdf_dir, "URS_PuertoRico.pdf"), pages=3, multiple_tables=True)
df_total = pd.DataFrame(tables_pg3[0])

df_total.columns

df_total = df_total[['Utilization', 
                    'State Number',
                    'State Rate']][0:3] ## only select the first 3 rows


display(df_total)

In [ ]:
# Define a function process_pdf to process raw pdf and save into a dataframe

def process_pdf(pdf_file, state_code): 

    ## Read table on page 3

    tables_pg3 = read_pdf(os.path.join(pdf_dir, pdf_file), pages=3, multiple_tables=True)
    
    ## Read tables[0] into a dataframe 
    df_total = pd.DataFrame(tables_pg3[0])
    df_total = df_total[['Utilization', 
                         'State Number',
                         'State Rate']][0:3] ## only select the first 3 rows

    ## Read tables from page 5 
    tables_pg5= read_pdf(os.path.join(pdf_dir,pdf_file), pages=5, multiple_tables=True )

    df_age = pd.DataFrame(tables_pg5[0])

    ## Only selecting specfic rows and columns that are associated with served patients & rates
    df_ageshort = df_age[["Unnamed: 0", "Unnamed: 1", "Unnamed: 5"]][4:13]

    ## Rename the df_ageshort dataframe by the column index
    df_2 = df_ageshort.rename(columns={df_ageshort.columns[0]: "Measure", 
                                       df_ageshort.columns[1]: "PatientsServed", 
                                       df_ageshort.columns[2]: "ServedRate"})



    ## Combine Total measure summary table with the age table 
    df_total = df_total.rename(columns= {df_total.columns[0]: "Measure", 
                                         df_total.columns[1]: "PatientsServed", 
                                         df_total.columns[2]: "ServedRate"})


    df_transformed = df_total.append(df_2)

    ## Add a column indicating the state 
    df_transformed['State'] = state_code

    return df_transformed




In [ ]:
## Testing on individual pdf 
tables_pg5= read_pdf(os.path.join(pdf_dir, 'URS_Utah.pdf'), pages=5, multiple_tables=True)

df_age = pd.DataFrame(tables_pg5[0])

## Only selecting specfic rows and columns that are associated with served patients & rates
df_ageshort = df_age[["Unnamed: 0", "Unnamed: 1", "Unnamed: 5"]][4:13]

## Rename the df_ageshort dataframe by the column index
df_2 = df_ageshort.rename(columns={df_ageshort.columns[0]: "Measure", 
                                       df_ageshort.columns[1]: "PatientsServed", 
                                       df_ageshort.columns[2]: "ServedRate"})

display(df_2)

In [ ]:
# Check if the output dir exists, if not, creat a dir
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)

# Create an empty list to save all the state-specific dataframe
all_transformed_data = []
# Loop through all PDF files in the PDF dir
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"): 
        state_code = pdf_file.split('_')[1].split('.')[0] # extract the state code from the file name
        df_transformed = process_pdf(pdf_file, state_code)
        print(df_transformed.head(20))
        all_transformed_data.append(df_transformed)
    
    

# Combine all states into a single dataframe
combined_df = pd.concat(all_transformed_data, ignore_index=True)

display(combined_df)


In [ ]:
# Investigate the number of states and territorities 
combined_df.State.nunique()

combined_df.State.unique()



In [ ]:

## Save the combined dataframe into a excel file 
combined_df.to_excel(os.path.join(output_dir, 'AllStates_2022.xlsx'),
                     startcol=0, startrow=0,
                     header=True, 
                     sheet_name='All States Measures')